1. So, I need a comprehensive set of movies and all the people that worked for it. My way to do it would be to split the df vertically into citing movie info df and cited movie df.

2. Then concatenate those, check for doubles and check if they have the same people working on the team or there are discrepancies.

3. Finally cancel replicates of movies, rows that are double basically.

In [12]:
import os
import pandas as pd

# creating the tables with pandas to interact with our datasets

#name_basics = pd.read_csv("../Dataset/name.basics.tsv" , sep='\t', header=0)
crew_data = pd.read_csv("../Dataset/crew_data.csv" , header=0)

citing_movies = crew_data[["tconst_movie", "movie", "movie_country", "year", "titleType", "tconst_cit", "nconst_movie", "category_movie", "job_movie"]]
cited_movies = crew_data[["tconst_cit", "cit", "country_cit", "primaryTitle_cit", "year_cit", "titleType_cit", "nconst_cit", "category_cit", "job_cit"]]
#column_names = list(crew_data.columns.values)
#name_basics[:30]
#print(set(crew_data.category_cit.values))
#citing_movies.describe(include="all")
citing_movies.dtypes

# starting to filter dataset
citing_movies = citing_movies[citing_movies['titleType'] != 'short']
citing_movies['year'] = citing_movies['year'].astype("string")

#function to remove .0 in year column
def remove_dotzero(s):
    return s.replace(".0", "")

citing_movies["year"] = citing_movies["year"].apply(remove_dotzero)

#renaming the columns for mental order
#N.B. because I put inplace = True I don't need to reassing the renamed columns to a new df variable, the original df is being modified on the fly.
citing_movies.rename(columns = {'tconst_movie' : 'movie_ID', 'movie' : 'title', 'nconst_movie' : 'person_ID', 'category_movie'  : 'person_role', 'tconst_cit' : 'cites'}, inplace= True)

#drop titleType column
#citing_movies = citing_movies.drop(['titleType', 'job_movie'], axis=1)

#analysing the ds 
temp =  citing_movies.query("movie_ID == 'tt5611424'")
print(len(temp))
temp

#drop titleType column
citing_movies = citing_movies.drop(['titleType', 'job_movie'], axis=1)

temp

15


,movie_ID,title,movie_country,year,titleType,cites,person_ID,person_role,job_movie
43,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3106201,director,\N
44,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3106201,director,\N
45,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3106201,director,\N
46,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3106201,director,\N
47,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm4097598,writer,based on characters by
48,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm4097598,writer,based on characters by
49,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm4097598,writer,based on characters by
50,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm4097598,writer,based on characters by
51,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3359084,producer,producer
52,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3359084,producer,producer


I realize that the reason why the same movie is repeated so many times is because for every movie also the info about the movie they cite. So for every movie cited there will be three lines for the cited movie director, writer and producer.

I will just remove duplicate rows in the df

In [13]:
print(len(citing_movies))
citing_movies = citing_movies.drop_duplicates()
print(len(citing_movies))

citing_movies

1651011
336480


,movie_ID,title,movie_country,year,cites,person_ID,person_role
43,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm3106201,director
47,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm4097598,writer
51,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm3359084,producer
55,tt5611424,#FromJennifer (2017),USA,2017,tt2630300,nm3106201,director
56,tt5611424,#FromJennifer (2017),USA,2017,tt2630300,nm4097598,writer
...,...,...,...,...,...,...,...
1964814,tt1646974,Le nom des gens (2010),France,2010,tt1068646,nm1928581,writer
1964820,tt1646974,Le nom des gens (2010),France,2010,tt1068646,nm0012433,producer
1964826,tt1646974,Le nom des gens (2010),France,2010,tt1068646,nm1206845,producer
1964832,tt1646974,Le nom des gens (2010),France,2010,tt1068646,nm0717873,producer


In [14]:
#additional exploring of the df
BeKindRewind =  citing_movies.query("movie_ID == 'tt0799934'")
print(len(BeKindRewind))
BeKindRewind = BeKindRewind.drop_duplicates()
print(len(BeKindRewind))
BeKindRewind.describe(include="all")

citing_movies.describe(include="all")
BeKindRewind.describe(include="all")


4833
4833


,movie_ID,title,movie_country,year,cites,person_ID,person_role
count,4833,4833,4833,4833,4833,4833,4833
unique,1,1,3,1,537,3,2
top,tt0799934,Be Kind Rewind (2008),UK,2008,tt0417148,nm0327273,producer
freq,4833,4833,1611,4833,9,1611,3222


In [15]:
from pprint import pprint

# iterating over columns unique values to see what values are repeated
column_names = list(BeKindRewind.columns.values) #put headers in a list
print(column_names)
'''print(len(BeKindRewind.cites.values))
print(len(set(BeKindRewind.cites.values)))'''

i = 0
while i <= len(column_names)-1:
    print("\n\n all values of the " + column_names[i] + " column are: \n")
    print(BeKindRewind[column_names[i]].values)
    print(set(BeKindRewind[column_names[i]].values))
    print(len(set(BeKindRewind[column_names[i]].values)))
 
    i += 1

['movie_ID', 'title', 'movie_country', 'year', 'cites', 'person_ID', 'person_role']


 all values of the movie_ID column are: 

['tt0799934' 'tt0799934' 'tt0799934' ... 'tt0799934' 'tt0799934'
 'tt0799934']
{'tt0799934'}
1


 all values of the title column are: 

['Be Kind Rewind (2008)' 'Be Kind Rewind (2008)' 'Be Kind Rewind (2008)'
 ... 'Be Kind Rewind (2008)' 'Be Kind Rewind (2008)'
 'Be Kind Rewind (2008)']
{'Be Kind Rewind (2008)'}
1


 all values of the movie_country column are: 

['UK' 'UK' 'UK' ... 'USA' 'USA' 'USA']
{'UK', 'France', 'USA'}
3


 all values of the year column are: 

['2008' '2008' '2008' ... '2008' '2008' '2008']
{'2008'}
1


 all values of the cites column are: 

['tt0417148' 'tt0417148' 'tt0417148' ... 'tt0118147' 'tt0118147'
 'tt0118147']
{'tt0366920', 'tt0370263', 'tt0097428', 'tt0120531', 'tt8816716', 'tt0095742', 'tt0118880', 'tt0162346', 'tt0088011', 'tt0118583', 'tt0387131', 'tt0211142', 'tt0113492', 'tt0095031', 'tt0160797', 'tt0076164', 'tt15125364', 

So, repetitions occur because of:
1. members of crew
2. multiple countries of production

with Be kind rewind, the most frequent movie in our dataframe, whe have:

537 citations * 3 crew members * 3 countries of production = 4833 instances

### Now I proceed to clean the cited movies dataframe:

In [16]:
cited_movies = crew_data[["tconst_cit", "cit", "country_cit", "primaryTitle_cit", "year_cit", "titleType_cit", "nconst_cit", "category_cit", "job_cit"]]

# starting to filter dataset
cited_movies = cited_movies[cited_movies['titleType_cit'] != 'short']
cited_movies['year_cit'] = cited_movies['year_cit'].astype("string")

#function to remove .0 in year column
def remove_dotzero(s):
    return s.replace(".0", "")

cited_movies["year_cit"] = cited_movies["year_cit"].apply(remove_dotzero)

#renaming the columns as the citing_movie df
cited_movies.rename(columns = {'tconst_cit' : 'movie_ID', 'cit' : 'title', 'country_cit' : 'movie_country', 'year_cit' : 'year', 'nconst_cit' : 'person_ID', 'category_cit'  : 'person_role'}, inplace= True)

#N.B this df has no 'cite' column: for the movies that are already present in citing_movies, I can just drop the rows that are repeated (drop duplicates, keep first instance).
# but there might be cited movies that cite no film
#  

#drop titleType column
cited_movies = cited_movies.drop(['titleType_cit', 'job_cit', 'primaryTitle_cit'], axis=1)
print(len(cited_movies))
cited_movies = cited_movies.drop_duplicates()
print(len(cited_movies))

cited_movies

1732986
97349


,movie_ID,title,movie_country,year,person_ID,person_role
0,tt0078748,Alien (1979),UK,1979,nm0001353,producer
1,tt0078748,Alien (1979),UK,1979,nm0000631,director
2,tt0078748,Alien (1979),UK,1979,nm0639321,writer
3,tt0078748,Alien (1979),UK,1979,nm0795953,writer
4,tt0078748,Alien (1979),UK,1979,nm0140826,producer
...,...,...,...,...,...,...
1964809,tt1068646,Entre les murs (2008),France,2008,nm0134559,director
1964810,tt1068646,Entre les murs (2008),France,2008,nm0133028,writer
1964811,tt1068646,Entre les murs (2008),France,2008,nm0035993,producer
1964812,tt1068646,Entre les murs (2008),France,2008,nm0071379,producer


In [17]:
cited_movies.describe(include="all")

,movie_ID,title,movie_country,year,person_ID,person_role
count,97349,97349,97335,97349,96951,96951
unique,21453,20664,123,115,33429,3
top,tt0114863,Treasure Island (1972),USA,2002,nm0000636,writer
freq,54,75,42760,2134,116,45009


Now I need to find the intersection between citing_movies id and cited_movies.
I need to:
1. find out what movies in cited_movies are already present in citing_movies
2. check wether the information for same movie_ID are coherent
3. if they are, they are duplicates and I can just drop the cited_movie instance
4. else, inquire why the related info are different and eventually assing the cites value to the cited_movie instance.
5. for movies only present in the cited_movie df, leave them in the concatenated df with nan value in cites column

In [18]:
# First I want to save data about citations from the first dataframe
# I will do it just by creating a subdataframe with the relevant columns

citations_data = citing_movies[["movie_ID", "cites"]]
citations_data

,movie_ID,cites
43,tt5611424,tt4627224
47,tt5611424,tt4627224
51,tt5611424,tt4627224
55,tt5611424,tt2630300
56,tt5611424,tt2630300
...,...,...
1964814,tt1646974,tt1068646
1964820,tt1646974,tt1068646
1964826,tt1646974,tt1068646
1964832,tt1646974,tt1068646


In [19]:
# there are 16379 citing movies, 21453 cited movies and 5349 are in common but ofc in the dataframes are repeated
# Than I add cites column to cited movies
#cited_movies.insert(4, 'cites', pd.Series(dtype="string"))

# concatenating
concat_df = pd.concat([citing_movies, cited_movies])
#len(concat_df) #433829
concat_df.drop_duplicates(inplace=True)
#subset=['movie_ID', 'title', 'movie_country', 'year', 'person_ID', 'person_role']

len(concat_df) #142818


433829

Now I need to add the info about the people that I can retreive from name.basics.csv

In [20]:
people_data = pd.read_csv("../Dataset/name.basics.tsv", sep="\t", header=0)
people_data = people_data[["nconst", "primaryName", "birthYear", "deathYear"]]
people_data

network2data = concat_df.merge(people_data, left_on="person_ID", right_on="nconst", how="left") #should be 142818 rows still
network2data.to_csv("../Dataset/network2data.csv")
